<a href="https://colab.research.google.com/github/niss10/IS-698-FL-Project/blob/main/FedCollab_FL_latest_flower_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latest Flower implementation
This notebook uses latest flower code. So it is based on latest flower implementation

# Prerequisite - Environment Setup

1. Run the below cell
2. Then, RESTART the Colab runtime. (after succefull completion)
3. After Restarting don't run this cell

In [ ]:

# Force‑reinstall NumPy and scikit‑learn
!pip install --upgrade --force-reinstall numpy==1.24.3 scikit-learn

# Install the CPU‑only PyTorch build
!pip uninstall -y torch torchvision
!pip install --index-url https://download.pytorch.org/whl/cpu torch torchvision

# Install/latest versions of Flower and PyTorch
!pip install --upgrade flwr pandas
!pip install -U "flwr[simulation]"

# Then, RESTART the Colab runtime.

# After Restarting don't run this cell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully uninstalled threadpoolctl-3.6.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.0
    Uninstalling joblib-1.5.0:
      Successfully uninstalled joblib-1.5.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfu

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 98.9 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0+cpu which is incompatible.
torchaudio 2.6.0+cu12

## Import required libraries

In [ ]:
# Import required libraries
import flwr as fl
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import os
from flwr.server.strategy import FedAvg
import time
from flwr.common import Metrics
from flwr.client import Client, ClientApp, NumPyClient
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.simulation import run_simulation
from flwr.common import Context

## Adding Master file
Setting source of truth file that we will use for federated learning simulation

upload the downloaded preprocessed_dataset.csv from Dataset/Pre-Processed/ folder

In [ ]:
# Adding Master dataset file
print("Please upload preprocessed_dataset.csv")
uploaded = files.upload()  # upload the downloaded preprocessed_dataset.csv
if(uploaded):
  print("File uploaded successfully")
else:
  print("File upload failed, Please upload again")
  uploaded = files.upload()

Please upload preprocessed_dataset.csv


Saving preprocessed_dataset.csv to preprocessed_dataset.csv
File uploaded successfully


# Data Partitioning for Client simulations

In [ ]:
# Data Partitioning Utility

# Load the master preprocessed dataset.
def load_master(csv_path):
    return pd.read_csv(csv_path)

# Randomly select N different user from all
def sample_user_ids(df, num_clients, seed=42):
    user_ids = df["UserID"].unique()
    rng = np.random.default_rng(seed)
    if num_clients >= len(user_ids):
        return list(user_ids)
    return list(rng.choice(user_ids, size=num_clients, replace=False))

# Save each selected user's rating to out_dir/user_{uid}.csv
def partition_by_user(df, out_dir, user_ids=None):
    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)

    if user_ids is None:
        user_ids = df["UserID"].unique()

    csv_paths = []
    for uid in user_ids:
        df_user = df[df["UserID"] == uid]
        path = out_path / f"user_{uid}.csv"
        df_user.to_csv(path, index=False)
        csv_paths.append(str(path))
    return csv_paths

## Sanity check data of partioning

In [ ]:

# Load the master dataset that we have uploaded earlier
MASTER_CSV = "preprocessed_dataset.csv"
df_master  = load_master(MASTER_CSV)
print("Rows:", len(df_master), " | Users:", df_master['UserID'].nunique(), " | Movie:", df_master['MovieID'].nunique())

# Cheking for 5 clients
paths_5   = partition_by_user(df_master, "clients_5",   sample_user_ids(df_master, 5))


print(f"5-client num files:   {len(paths_5)}")
print(f"5-client files:   {paths_5}")


<ipython-input-3-2292d8e7e956>:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_path)


Rows: 1000209  | Users: 6040  | Movie: 3706
5-client num files:   5
5-client files:   ['clients_5/user_4673.csv', 'clients_5/user_2651.csv', 'clients_5/user_3953.csv', 'clients_5/user_539.csv', 'clients_5/user_2616.csv']


# Model & Local-Training Utilities
In this section we are difining model and how model will train data we have used REcommenderMLP using Pytorch for training.

## Loading Local data and Train/Test Split

In [ ]:
# Return train & test DataLoaders for one client CSV.
def load_client_data(csv_path, batch_size=32, test_size=0.2, seed=42):
    df = pd.read_csv(csv_path)
    users  = torch.tensor(df["UserID"].values,   dtype=torch.long)
    movies = torch.tensor(df["MovieID"].values,  dtype=torch.long)
    ratings= torch.tensor(df["Rating"].values,   dtype=torch.float32)

    idx = np.arange(len(df))
    rng = np.random.default_rng(seed)
    rng.shuffle(idx)
    split = int((1 - test_size) * len(idx))
    train_idx, test_idx = idx[:split], idx[split:]

    train_ds = TensorDataset(users[train_idx], movies[train_idx], ratings[train_idx])
    test_ds  = TensorDataset(users[test_idx],  movies[test_idx],  ratings[test_idx])

    return (DataLoader(train_ds, batch_size=batch_size, shuffle=True),
            DataLoader(test_ds,  batch_size=batch_size, shuffle=False))

## Model

In [ ]:
# Defining RecommenderMLP Model using Pytorch

# User-embedding + Movie-embedding -> 2-layer MLP[64 -> ReLu] -> rating (1-5).
class RecommenderMLP(nn.Module):
    def __init__(self, num_users, num_movies,
                 embed_dim=32, hidden=64):
        super().__init__()
        self.user_embed  = nn.Embedding(num_users  + 1, embed_dim)
        self.movie_embed = nn.Embedding(num_movies + 1, embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim * 2, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, user_ids, movie_ids):
        u = self.user_embed(user_ids)
        m = self.movie_embed(movie_ids)
        x = torch.cat([u, m], dim=1)
        return self.mlp(x).squeeze(1)

## Training and Evaluation

In [ ]:
# Utility for loading data into tensor for client and training and evalation

# this function will train Model for one epoch
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total, count = 0.0, 0
    for u, m, r in loader:
        u, m, r = u.to(device), m.to(device), r.to(device)
        optimizer.zero_grad()
        loss = criterion(model(u, m), r)
        loss.backward()
        optimizer.step()
        total += loss.item() * len(r)
        count += len(r)
    return total / count

# Evaluate train model on test dataset and return loss
def evaluate(model, loader, criterion, device):
    model.eval()
    total, count = 0.0, 0
    with torch.no_grad():
        for u, m, r in loader:
            u, m, r = u.to(device), m.to(device), r.to(device)
            loss = criterion(model(u, m), r)
            total += loss.item() * len(r)
            count += len(r)
    return total / count

## Quick Sanity Check of Model

In [ ]:
# running model for 1 user from 5 client we have created to check how it works
sample_csv = "clients_5/" + sorted(os.listdir("clients_5"))[0]

# Get dataset dimensions for embeddings
num_users  = df_master["UserID"].max()
num_movies = df_master["MovieID"].max()

# Build loaders and model
train_loader, test_loader = load_client_data(sample_csv, batch_size=16)
device = torch.device("cpu")
model  = RecommenderMLP(num_users, num_movies).to(device)

opt  = torch.optim.Adam(model.parameters(), lr=1e-3)
crit = torch.nn.MSELoss()

Train_loss = train_one_epoch(model, train_loader, opt, crit, device)
Test_loss  = evaluate(model, test_loader, crit, device)

print("Train loss:", Train_loss)
print("Test  loss:", Test_loss)

Train loss: 12.05018367767334
Test  loss: 12.613490104675293


# FL Setup

## FL Client

In [ ]:
# Flower Client
class FLClient(NumPyClient):
    def __init__(self, model, train_loader, test_loader, device):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.device = device
        self.criterion = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def get_parameters(self, config):
        return [p.cpu().numpy() for p in self.model.state_dict().values()]

    def set_parameters(self, params):
        keys = list(self.model.state_dict().keys())
        self.model.load_state_dict({k: torch.tensor(v) for k, v in zip(keys, params)}, strict=True)

    def fit(self, params, config):
        self.set_parameters(params)
        num_epochs = CONFIG["num_epochs"]
        for _ in range(num_epochs):
           local_train_loss = train_one_epoch(self.model, self.train_loader, self.optimizer, self.criterion, self.device)
           print("Local training loss:", local_train_loss)
        return self.get_parameters(config), len(self.train_loader.dataset), {}

    def evaluate(self, params, config):
        self.set_parameters(params)
        loss = evaluate(self.model, self.test_loader, self.criterion, self.device)
        return float(loss), len(self.test_loader.dataset), {}

# Client Function or Client App
def client_fn(context: Context) -> Client:
    idx = int(context.node_config["partition-id"])
    csv  = csv_paths[idx] # csv file per user that we have created earlier
    client_training_data, client_test_data = load_client_data(csv, batch_size=32) # Creating training data and test data from those per user/clent csv file
    model  = RecommenderMLP(num_users, num_movies, CONFIG["embed_dim"], CONFIG["hidden_dim"])
    return FLClient(model, client_training_data, client_test_data, device=torch.device(CONFIG["device"])).to_client()

## FL Server


In [ ]:
# Server Function or Server App which has strategy how to FL will run
def server_fn(context: Context) -> ServerAppComponents:
    strategy = FedAvg(
        fraction_fit=1,
        fraction_evaluate=1,
        min_fit_clients=CONFIG["num_clients"],
        min_evaluate_clients=CONFIG["num_clients"],
        min_available_clients=CONFIG["num_clients"],
    )
    config = ServerConfig(num_rounds=CONFIG["num_rounds"])
    return ServerAppComponents(strategy=strategy, config=config)

## Federated Runner

In [ ]:

def run_federated():
  global csv_paths, num_users, num_movies # we need these in Client Function
  df_master = load_master(CONFIG["data_path"])
  selected_uids = sample_user_ids(df_master, CONFIG["num_clients"])
  folder = f"clients_{CONFIG['num_clients']}"
  csv_paths = partition_by_user(df_master, folder, selected_uids)

  # Set dataset dimensions
  num_users = int(df_master["UserID"].max())
  num_movies = int(df_master["MovieID"].max())

  # Defining client and server app
  client_app = ClientApp(client_fn=client_fn)
  server_app = ServerApp(server_fn=server_fn)

  # Run simulation
  start_time = time.perf_counter()
  run_simulation(
            client_app=client_app,
            server_app=server_app,
            num_supernodes=CONFIG["num_clients"],
            backend_config={
                "client_resources": {
                    "num_cpus": 1,
                }
            },
        )
  end_time = time.perf_counter()
  print(f"Time taken: {end_time - start_time:0.4f} seconds")
  # for r, mse in history.losses_distributed:
  #       print(f"Round {r:02d}: MSE={mse:.4f}, RMSE={np.sqrt(mse):.4f}")
  #       mse_values.append(mse)
  #       rmse_values.append(np.sqrt(mse))

  # return history, mse_values, rmse_values

## Triggering Federated Learning

In [ ]:
# Configuration
CONFIG = {
    "num_clients": 100,
    "num_rounds": 10,
    "num_epochs": 3,
    "data_path": "preprocessed_dataset.csv",
    "device": "cpu",
    "embed_dim": 32,
    "hidden_dim": 64,
}

run_federated()
# history, mse_values, rmse_values   = run_federated()

<ipython-input-3-2292d8e7e956>:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_path)
DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=10, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=10856) 2025-05-16 18:53:15.313415: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=10856) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=10856) E0000 00:00:1747421595.369562   10856 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=10856) E0000 00:00:1747421595.383326   10856 cuda_blas.cc:1418] Unable to register cuBLAS factory: 

(ClientAppActor pid=10856) Local training loss: 16.653217315673828
(ClientAppActor pid=10856) Local training loss: 16.112255096435547
(ClientAppActor pid=10856) Local training loss: 15.579436302185059
(ClientAppActor pid=10856) Local training loss: 14.119012045628816
(ClientAppActor pid=10856) Local training loss: 11.912876888386254
(ClientAppActor pid=10856) Local training loss: 9.843495961531852
(ClientAppActor pid=10856) Local training loss: 15.991059590899756
(ClientAppActor pid=10856) Local training loss: 14.24217476920476
(ClientAppActor pid=10856) Local training loss: 12.525372262984988
(ClientAppActor pid=10856) Local training loss: 17.235469818115234
(ClientAppActor pid=10856) Local training loss: 16.67262077331543
(ClientAppActor pid=10856) Local training loss: 16.12274932861328
(ClientAppActor pid=10856) Local training loss: 11.27953815460205
(ClientAppActor pid=10856) Local training loss: 10.405283110482353
(ClientAppActor pid=10856) Local training loss: 9.575954982212611
(

INFO :      aggregate_fit: received 100 results and 0 failures
INFO :      configure_evaluate: strategy sampled 100 clients (out of 100)
INFO :      aggregate_evaluate: received 100 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 100 clients (out of 100)


(ClientAppActor pid=10856) Local training loss: 7.1552000859888585 [repeated 61x across cluster]
(ClientAppActor pid=10855) Local training loss: 10.489712380074167 [repeated 40x across cluster]
(ClientAppActor pid=10856) Local training loss: 1.8214378176110515 [repeated 79x across cluster]
(ClientAppActor pid=10856) Local training loss: 9.572186946868896 [repeated 51x across cluster]
(ClientAppActor pid=10856) Local training loss: 7.606056232263546 [repeated 63x across cluster]


INFO :      aggregate_fit: received 100 results and 0 failures
INFO :      configure_evaluate: strategy sampled 100 clients (out of 100)
INFO :      aggregate_evaluate: received 100 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 100 clients (out of 100)


...

(ClientAppActor pid=10855) Local training loss: 0.7277721977233886 [repeated 100x across cluster]
(ClientAppActor pid=10855) Local training loss: 1.3910060447195303 [repeated 127x across cluster]
(ClientAppActor pid=10855) Local training loss: 1.1617736233605278 [repeated 125x across cluster]


INFO :      aggregate_fit: received 100 results and 0 failures
INFO :      configure_evaluate: strategy sampled 100 clients (out of 100)
INFO :      aggregate_evaluate: received 100 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 224.20s
INFO :      	History (loss, distributed):
INFO :      		round 1: 9.947847074384526
INFO :      		round 2: 7.648478051608073
INFO :      		round 3: 5.834322568867449
INFO :      		round 4: 4.4725045909434105
INFO :      		round 5: 3.517450380472675
INFO :      		round 6: 2.851048617322077
INFO :      		round 7: 2.3912904856258517
INFO :      		round 8: 2.080417061982043
INFO :      		round 9: 1.8696711163302484
INFO :      		round 10: 1.7279042162858214
INFO :      


(ClientAppActor pid=10855) Local training loss: 1.1007659718932876 [repeated 45x across cluster]


(pid=10855) 2025-05-16 18:53:15.312993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=10855) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=10855) E0000 00:00:1747421595.368175   10855 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=10855) E0000 00:00:1747421595.383369   10855 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Time taken: 252.3693 seconds
